In [ ]:
#pip install openai
#pip install langchain
#pip install chromadb
#pip install tiktoken
#pip install pypdf
#pip install gradio
#pip install unstructured
#pip install "unstructured[all-docs]"

# Importing necessary libraries

In [ ]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
import constants
import os
from langchain.chat_models import ChatOpenAI

# Importing OpenAI-API key and defining the cahtbot modell

In [ ]:
os.environ["OPENAI_API_KEY"] = constants.APIKEY
llm = ChatOpenAI(temperature=0, model="gpt-4-0613")

In [ ]:
import PIL
print(PIL.__version__)

# Loading the data from local directory

In [ ]:
from langchain.document_loaders import DirectoryLoader

pdf_loader = DirectoryLoader('./Files/', glob="**/*.pdf")

loaders = [pdf_loader]
documents = []
for loader in loaders:
    documents.extend(loader.load())

print(f"Total number of documents: {len(documents)}")

# Chunking the data, creations Embeddings and store those in VectorStore

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)

# Using the Langchain's Chat with Chat History

In [ ]:
qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(temperature=0), vectorstore.as_retriever(), return_source_documents=True)

# Implementation Gradio-WebApp

In [ ]:
import gradio as gr

with gr.Blocks() as app:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")
    chat_history = []
    

    def user(user_message, history):
        print("User message:", user_message)
        print("Chat history", history)

        # Convert chat history to a list of tuples
        chat_history_tuples = [(item[0], item[1]) for item in history]

        # Get response from QA chain
        response = qa({"question": user_message, "chat_history": chat_history_tuples})
        history.append((user_message, response["answer"]))
        print("Updated chat history:", history)
        return gr.update(value=""),history
    
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)

    clear.click(lambda: None, None, chatbot, queue=False)



if __name__ == "__main__":
    app.launch(debug=True)